# All notebooks in one

In [1]:
TIMES = dict()
RELATIVE_SPEEDUP = dict()
LOOPS_DEFAULT = 100/1
ITERATIONS_DEFAULT = 20000/1
NUMBER_DEFAULT = 1
REPEATS_DEFAULT = 1
CANARY_MODE_DEFAULT = False

In [2]:
"""
import timeit
def measure_file_old(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT, orig_globals=orig_globals):
    with open("nbody_%s.py" % file_suffix,'r') as f:
        total_statement = f.read()
    total_statement = total_statement.replace("nbody(100, 'sun', 20000)", "nbody(%i, 'sun', %i)" % (loops, iterations))
    total_statement = total_statement.replace("print", "")
    total_statement = "print('running')\n" + total_statement + "\n    print('done')" 
    with open("nbody_%s.canary.py" % file_suffix, 'w') as f:
        f.write(total_statement)
    return 999
    #times = [timeit.timeit(stmt=total_statement, number=3, globals=orig_globals) for it in range(number)]
    #return float(sum(times)) / float(len(times))
"""

'\nimport timeit\ndef measure_file_old(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT, orig_globals=orig_globals):\n    with open("nbody_%s.py" % file_suffix,\'r\') as f:\n        total_statement = f.read()\n    total_statement = total_statement.replace("nbody(100, \'sun\', 20000)", "nbody(%i, \'sun\', %i)" % (loops, iterations))\n    total_statement = total_statement.replace("print", "")\n    total_statement = "print(\'running\')\n" + total_statement + "\n    print(\'done\')" \n    with open("nbody_%s.canary.py" % file_suffix, \'w\') as f:\n        f.write(total_statement)\n    return 999\n    #times = [timeit.timeit(stmt=total_statement, number=3, globals=orig_globals) for it in range(number)]\n    #return float(sum(times)) / float(len(times))\n'

In [3]:
import timeit
import numpy as np
from subprocess import check_output

def measure_file(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT
                 , repeat=REPEATS_DEFAULT, canary_mode=CANARY_MODE_DEFAULT):
    with open("nbody_%s.py" % file_suffix,'r') as f:
        total_statement = f.read()
    
    
    total_statement = total_statement.replace("nbody(100, 'sun', 20000)", "nbody(%i, 'sun', %i)" % (loops, iterations))
    if canary_mode:
        total_statement = total_statement.replace("print", "")
        total_statement = "print('running')\n" + total_statement + "\n    print('done')" 
    
    
    with open("nbody_%s.canary.py" % file_suffix, 'w') as f:
        f.write(total_statement)
    
    times = timeit.repeat('check_output(["python", "nbody_%s.canary.py"])' % file_suffix,
                         setup='from subprocess import check_output',
                         number=number,
                         repeat=repeat)
    return np.array(times).mean() /number
        

In [4]:
if False: 
    name = 'orig'
    TIMES[name] = measure_file(name)
    RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
    TIMES[name], RELATIVE_SPEEDUP[name]

## Original

In [5]:
%%file nbody_orig.py
"""
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Writing nbody_orig.py


In [6]:
!cp nbody.py nbody_orig.py

In [7]:
name = 'orig'
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(80.408898497000337, 1.0)

In [8]:
!rm nbody_orig.py

# Problem 1
Reducing function call overhead

In [9]:
%%file nbody_1.py
"""The biggest change was removing the duplicate computations of compute_b(),
which was recycled 3 times more than it should have.
Inisde of compute_b() was compute_m(), and that was being replicated 6 times.

I also got rid of the update_* functions, which helped just a little.

Also, removed the duplicate and nonfunctional report_energy()

"""
"""
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

#def compute_deltas(x1, x2, y1, y2, z1, z2):
#    return (x1-x2, y1-y2, z1-z2)
    
#def compute_b(m, dt, dx, dy, dz):
#    mag = compute_mag(dt, dx, dy, dz)
#    return m * mag

#def compute_mag(dt, dx, dy, dz):
#    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

#def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
#    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
#    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
#    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
#    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
#    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
#    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

#def update_rs(r, dt, vx, vy, vz):
#    r[0] += dt * vx
#    r[1] += dt * vy
#    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

                #update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                
                mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
                
                thing_for_v1 = m2 * mag
                v1[0] -= dx * thing_for_v1
                v1[1] -= dy * thing_for_v1
                v1[2] -= dz * thing_for_v1
                
                thing_for_v2 = m1 * mag            
                v2[0] += dx * thing_for_v2
                v2[1] += dy * thing_for_v2
                v2[2] += dz * thing_for_v2
                
                
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        #update_rs(r, dt, vx, vy, vz)
        r[0] += dt * vx
        r[1] += dt * vy
        r[2] += dt * vz

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

                #e -= compute_energy(m1, m2, dx, dy, dz)
                e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())


if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_1.py


In [10]:
name=1
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(30.994290845002979, 2.5943132204285995)

# Problem 2

using alternatices to membership testing of lists

In [11]:
%%file nbody_2.py

"""
This made very little difference.

The biggest change I made was iterating through the body pairs using itertools.combinations( <body_name_list>, 2).

It also helped to store this result as an available variable.
"""

"""
    N-body simulation.
"""
from itertools import combinations

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}
VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    
    for (body1, body2) in VISIT_SCHEDULE:
        ([x1, y1, z1], v1, m1) = BODIES[body1]
        ([x2, y2, z2], v2, m2) = BODIES[body2]
        (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
                    
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]
        (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        
        e -= compute_energy(m1, m2, dx, dy, dz)
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_2.py


In [12]:
name = 2
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(69.164767191978171, 1.162570218357162)

# Problem 3

using local rather than global variables

In [13]:
%%file nbody_3.py
"""

I gave all functions references to the varaible BODIES which I used several times.  
However, this didn't help very much at all.

    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt, BODIES=BODIES):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)


def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0, BODIES=BODIES):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy(BODIES=BODIES)
        for _ in range(iterations):
            advance(0.01, BODIES=BODIES)
        print(report_energy(BODIES=BODIES))


if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_3.py


In [14]:
name = 3
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(78.692762500955723, 1.0218080537714478)

In [15]:
A = 1

def f(A=A):
    A += 1
    print("f = %s" % A)
    
def g(A=A):
    A += 1
    print("g = %s" % A)
    
f()
g()

f = 2
g = 2


In [16]:
A = dict()

def f(A=A):
    A['a'] = 'a'
    print("f = %s" % A)
    
def g(A=A):
    A['b'] = 'b'
    print("g = %s" % A)
    
f()
g()

f = {'a': 'a'}
g = {'b': 'b', 'a': 'a'}


# Problem 4
using data aggregation ro reduce loop overhead

In [17]:
%%file nbody_4.py
"""
    The biggest change here, by far, was using map() for the inner and outer loop of nbody()
    
    I also changed most of the other for loops to use map() instead.  This made only marginal differences.
    
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    def i_func(_):
        advance(0.01)
    
    def l_func(_):
        report_energy()
        list(map(i_func, range(iterations)))
        print(report_energy())
                                    
                                    
    list(map(l_func, range(loops)))

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_4.py


In [18]:
name = 4
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(80.421980100974906, 0.99983733795216001)

# opt

In [44]:
%%file nbody_opt.py
"""
    By far, the biggest change was removing the duplicated function calls.
    
    I tried parallelization for the loops where it was safe to do them, but it was actually slower.
    
    The relative speedups were:
    {1: 2.3767373358321096,
    2: 1.1505620539678709,
    3: 1.0679881822025878,
    4: 2049.8051039167353,
    'orig': 1.0,
    'opt': 2281.8479520841111}
    
    
    The times were:
    {1: 33.572911055001896,
    2: 69.352183918992523,
    3: 74.714301624975633,
    4: 0.038927599031012505,
    'orig': 79.793991176993586,
    'opt': 0.034969021973665804}
 
 
    So, yes, it runs in well under 30 seconds :-)
     
     
    N-body simulation.
"""
from itertools import combinations


PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

#def compute_deltas(x1, x2, y1, y2, z1, z2):
#    return (x1-x2, y1-y2, z1-z2)
    
#def compute_b(m, dt, dx, dy, dz):
#    mag = compute_mag(dt, dx, dy, dz)
#    return m * mag

#def compute_mag(dt, dx, dy, dz):
#    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

#def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
#    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
#    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
#    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
#    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
#    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
#    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

#def update_rs(r, dt, vx, vy, vz):
#    r[0] += dt * vx
#    r[1] += dt * vy
#    r[2] += dt * vz


def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    thing_for_v1 = m2 * mag
    v1[0] -= dx * thing_for_v1
    v1[1] -= dy * thing_for_v1
    v1[2] -= dz * thing_for_v1

    thing_for_v2 = m1 * mag            
    v2[0] += dx * thing_for_v2
    v2[1] += dy * thing_for_v2
    v2[2] += dz * thing_for_v2

def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    #update_rs(r, dt, vx, vy, vz)
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
def advance(dt=dt, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        advance the system one timestep
    '''
    
    list(map(to_do, BODIES1, BODIES2))      
    list(map(update_rs_for_body, BODIES_KEYS))
    

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]
        #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        #e -= compute_energy(m1, m2, dx, dy, dz)
        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    #for body, (r, [vx, vy, vz], m) in BODIES.items():
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

        
    for _ in range(loops):
        for _ in range(iterations):            
            list(map(to_do, BODIES1, BODIES2))      
            list(map(update_rs_for_body, BODIES_KEYS))
            #advance(0.01, BODIES=BODIES)
        print(report_energy(BODIES=BODIES))
            
    """    
    def i_func(_):
        advance()
    
    def l_func(_):
        list(map(i_func, range(iterations)))
        print(report_energy())
                                    
                                    
    list(map(l_func, range(loops)))
"""

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_opt.py


In [45]:
name = 'opt'
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(28.600031132984441, 2.8114968869479546)

In [46]:
RELATIVE_SPEEDUP

{1: 2.5943132204285995,
 2: 1.162570218357162,
 3: 1.0218080537714478,
 4: 0.99983733795216001,
 'orig': 1.0,
 'opt': 2.8114968869479546}

In [47]:
did_i_break_30 = TIMES['opt'] < 30

did_i_break_30 = "I did break 30 seconds. :)" if did_i_break_30 else "I did not break 30 seconds. :("
did_i_break_30

'I did break 30 seconds. :)'

In [48]:
with open("nbody_opt.py", 'r') as f:
    text = f.read()
    
with open("nbody_opt.py", 'w') as f:
    text = ("""\n 
    \"\"\" 
    
    The relative speedups are: 
    %s
    
    The time it took to run is:
    %s
    
    %s
    
    \"\"\"\n
    """ % (RELATIVE_SPEEDUP, TIMES, did_i_break_30)
           ) + text
    
    f.write(text)

In [49]:
TIMES

{1: 30.994290845002979,
 2: 69.164767191978171,
 3: 78.692762500955723,
 4: 80.421980100974906,
 'orig': 80.408898497000337,
 'opt': 28.600031132984441}

In [50]:
!rm *.canary.py